In [ ]:
import json
import requests
import pandas as pd
from dotenv import load_dotenv
import os
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry


In [ ]:
load_dotenv()
OPENSEA_API = os.getenv('OPENSEA_API_KEY')

In [ ]:
def requests_session_with_retries():
    session = requests.Session()
    retries = Retry(total=5,  # Total number of retries to allow.
                    backoff_factor=1,  # A backoff factor to apply between attempts.
                    status_forcelist=[500, 502, 503, 504])  # A set of HTTP status codes that we should force a retry on.
    adapter = HTTPAdapter(max_retries=retries)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    return session

In [ ]:
def fetch_api_data_paginated(base_url, headers=None, data_key='asset_events'):
    session = requests_session_with_retries()
    all_data = []
    url = base_url
    
    while url:
        print(f"Requesting URL: {url}")
        response = session.get(url, headers=headers, timeout=30)
       
        if response.status_code == 200:
            data = response.json()
            page_data = data.get(data_key, [])
            all_data.extend(page_data)
            next_cursor = data.get('next', "")
            print(next_cursor)
            if next_cursor:
                url = f"{base_url}?next={next_cursor}"
            else:
                url = None
        else:
            print(f"Failed to fetch data from {url} - Status Code: {response.status_code}")
            break
        
    return pd.json_normalize(all_data)  # Convert the aggregated data to a DataFrame


In [ ]:
# import requests
# import pandas as pd
# 
# def requests_session_with_retries():
#     session = requests.Session()
#     retries = Retry(total=5, backoff_factor=1, status_forcelist=[500, 502, 503, 504])
#     adapter = HTTPAdapter(max_retries=retries)
#     session.mount('http://', adapter)
#     session.mount('https://', adapter)
#     return session
# 
# def fetch_api_data_paginated(base_url, headers=None, after=None, event_type=None, data_key='asset_events'):
#     session = requests_session_with_retries()
#     all_data = []
#     params = {
#         "after": after,  # Add the 'after' timestamp to the query parameters if provided
#         "event_type": event_type  # Add the 'event_type' to the query parameters if provided
#     }
#     counter = 0
# 
#     while counter <= 100:
#         print(f"Requesting URL: {base_url} with params: {params}")
#         response = session.get(base_url, headers=headers, params=params, timeout=30)
# 
#         if response.status_code == 200:
#             data = response.json()
#             page_data = data.get(data_key, [])
#             all_data.extend(page_data)
#             next_cursor = data.get('next', None)
#             print("Going to next page")
#             if next_cursor:
#                 params['cursor'] = next_cursor  # Update the 'cursor' in params for pagination
#             else:
#                 break  # Exit loop if there's no 'next' cursor
#         else:
#             print(f"Failed to fetch data from {base_url} - Status Code: {response.status_code}")
#             break
#         counter += 1
# 
#     return pd.json_normalize(all_data)  # Convert the aggregated data to a DataFrame
# 
# # Example usage
# headers = {
#     "accept": "application/json",
#     "x-api-key": "29b6c9b1120748878ac7f50821ec4d0b"
# }
# base_url = "https://api.opensea.io/api/v2/events/collection/collection_slug"  # Replace with your actual URL
# after_timestamp = "1672531200"  # Replace with your actual timestamp or None
# event_type = None  # Specify an event type if desired
# 
# # Fetching the data
# data_df = fetch_api_data_paginated(base_url, headers, after=after_timestamp, event_type=event_type)
# print(data_df)


In [ ]:
# Function to fetch data from an API
def fetch_api_data(url, headers=None):
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        df = pd.json_normalize(data, max_level=1)
        return df
    else:
        print(f"Failed to fetch data from {url}")
        return {}


In [ ]:
# USER CHOOSES COLLECTION
collection_slug = input("Enter the collection: ")
APR = input("Enter your target APR: ")



# API's
get_collection_stats = f"https://api.opensea.io/api/v2/collections/{collection_slug}/stats"
get_collection = f"https://api.opensea.io/api/v2/collections/{collection_slug}"
get_events_by_collection = f"https://api.opensea.io/api/v2/events/collection/{collection_slug}"
get_nft_by_collection = f"https://api.opensea.io/api/v2/collection/{collection_slug}/nfts"
get_traits = f"https://api.opensea.io/api/v2/traits/{collection_slug}"

headers = {"accept": "application/json", "x-api-key": OPENSEA_API}

collection_stats_df = fetch_api_data(get_collection_stats, headers)
collection_df = fetch_api_data(get_collection, headers)
traits_df = fetch_api_data(get_traits, headers)

# events_df = fetch_api_data_paginated(get_events_by_collection, headers, data_key='asset_events')
nft_df = fetch_api_data_paginated(get_nft_by_collection, headers, data_key='nfts')


In [ ]:
# print(events_df)
print(nft_df)


In [ ]:
# Process and convert data from the first API
df1_total = pd.DataFrame([collection_stats_df['total']])
df1_intervals = pd.DataFrame(collection_stats_df['intervals'])

In [ ]:
# Process and convert data from the second API
collection_fields = ['collection', 'name', 'description', 'image_url', 'banner_image_url', 'owner', 'category', 'is_disabled', 'is_nsfw', 'trait_offers_enabled', 'collection_offers_enabled', 'opensea_url', 'project_url', 'wiki_url', 'discord_url', 'telegram_url', 'twitter_username', 'instagram_username', 'total_supply', 'created_date']
df2_collection = pd.DataFrame([{field: collection_df.get(field, '') for field in collection_fields}])
df2_contracts = pd.DataFrame(collection_df.get('contracts', []))
df2_fees = pd.DataFrame(collection_df.get('fees', []))
df2_payment_tokens = pd.DataFrame(collection_df.get('payment_tokens', []))
df2_rarity = pd.DataFrame([collection_df.get('rarity', {})])

In [ ]:
events_normalized = []
for event in events_df['asset_events']:
    # Initialize event_data with event fields, excluding nft, asset, payment, criteria
    event_data = {k: v for k, v in event.items() if k not in ['nft', 'asset', 'payment', 'criteria']}
    
    # Check and add nft data if present
    if 'nft' in event and event['nft'] is not None:
        nft_data = {'nft_' + k: v for k, v in event['nft'].items()}
        event_data.update(nft_data)
    
    # Check and add asset data if present
    elif 'asset' in event and event['asset'] is not None:
        asset_data = {'asset_' + k: v for k, v in event['asset'].items()}
        event_data.update(asset_data)
    
    # Check and include payment data if necessary and present
    if 'payment' in event and event['payment'] is not None:
        payment_data = {'payment_' + k: v for k, v in event['payment'].items()}
        event_data.update(payment_data)
    
    events_normalized.append(event_data)

df_events = pd.DataFrame(events_normalized)

In [ ]:
# Export to CSV
df1_total.to_csv('api1_total_stats.csv', index=False)
df1_intervals.to_csv('api1_intervals_stats.csv', index=False)
df2_collection.to_csv('api2_collection_info.csv', index=False)
df2_contracts.to_csv('api2_collection_contracts.csv', index=False)
df2_fees.to_csv('api2_collection_fees.csv', index=False)
df2_payment_tokens.to_csv('api2_collection_payment_tokens.csv', index=False)
df2_rarity.to_csv('api2_collection_rarity.csv', index=False)
df_events.to_csv('api3_events.csv', index=False)


In [ ]:
print(df_events.head())

In [ ]:
print(df1_total.head())

In [ ]:
print(df1_intervals.head())

In [ ]:
print(df2_fees)

In [ ]:
print(df2_rarity)


In [ ]:
print(df2_payment_tokens)


In [ ]:
print(df2_collection)


In [ ]:
print(df2_contracts)
